In [8]:
import os 
import json
import pandas as pd
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI

In [10]:
pip install langchain_community


Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.chat_models import ChatOpenAI

In [11]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [12]:
KEY=os.getenv("OPENAI_API_KEY")

In [14]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

C:\Users\vijen\AppData\Local\Temp\ipykernel_1784\1961887400.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)


In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [18]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

""" 

In [19]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [20]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\vijen\AppData\Local\Temp\ipykernel_1784\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [21]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [23]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [24]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [26]:
file_path=r"C:\Users\vijen\mcqgen-v1\data.txt"

In [27]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [29]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [30]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [32]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\vijen\AppData\Local\Temp\ipykernel_1784\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [33]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1925
Prompt Tokens:1192
Completion Tokens:733
Total Cost:0.003254


In [34]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [35]:
quiz=response.get("quiz")

In [42]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Physics',
   'b': 'Chemistry',
   'c': 'Biology',
   'd': 'Geology'},
  'correct': 'c'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Cellular respiration',
   'd': 'Genetics'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Cell division',
   'c': 'Mutation',
   'd': 'Photosynthesis'},
  'correct': 'a'},
 '4': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '5': {'mcq': 'How long ago did life on Earth emerge?',
  'options': {'a': '1 billion years ago',
   'b': '2.5 billion years ago',
   'c': '3.7 billion years ago',
   'd': '5 billion years ago'},
  'correct': 'c'}}

In [43]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Physics | b: Chemistry | c: Biology | d: Geology',
  'Correct': 'c'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Cellular respiration | d: Genetics',
  'Correct': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Cell division | c: Mutation | d: Photosynthesis',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that processes hereditary information?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'Correct': 'c'},
 {'MCQ': 'How long ago did life on Earth emerge?',
  'Choices': 'a: 1 billion years ago | b: 2.5 billion years ago | c: 3.7 billion years ago | d: 5 billion years ago',
  'Correct': 'c'}]

In [47]:
quiz=pd.DataFrame(quiz_table_data)

In [48]:
quiz.to_csv("machinelearning.csv",index=False)

In [50]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_10_2024_10_00_49'